In [0]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, Dense, Dropout, Flatten, Bidirectional,TimeDistributed

In [2]:
df_train = pd.read_csv('../content/drive/My Drive/train.csv')
df_train

,time,signal,open_channels
0,0.0001,-2.7600,0
1,0.0002,-2.8557,0
2,0.0003,-2.4074,0
3,0.0004,-3.1404,0
4,0.0005,-3.1525,0
...,...,...,...
4999995,499.9996,2.9194,7
4999996,499.9997,2.6980,7
4999997,499.9998,4.5164,8
4999998,499.9999,5.6397,9


In [0]:
signal = np.array(df_train.signal)
open_channels = np.array(df_train.open_channels)
#signals = np.reshape(signal,(1))
data = np.zeros((len(signal), 2))
data[:, 0] = signal
data[:, 1] = open_channels
data = data.reshape((1000, 5000, 2))
np.random.shuffle(data)
x = data[:,:, 0].reshape((1000,5000,1))
y = data[:,:, 1].reshape((1000,5000,1))

In [0]:
model = Sequential()
model.add(Bidirectional(LSTM(units=20, return_sequences=True, input_shape=(5000,1))))
# model.add(Dense(20,activation='relu'))
# model.add(Dense(units=1,activation='relu'))
model.add(TimeDistributed(Dense(units=1,activation='relu')))
model.compile(loss='mean_squared_error',optimizer='sgd',metrics=['accuracy'])

In [0]:
# train LSTM
model.fit(x, y, epochs=10, validation_split=.3, batch_size=1, verbose=1)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

700/700 [==============================] - 181s 258ms/step - loss: 3.1364 - accuracy: 0.3637 - val_loss: 2.6512 - val_accuracy: 0.3937
Epoch 2/10
700/700 [==============================] - 182s 260ms/step - loss: 2.9206 - accuracy: 0.3861 - val_loss: 2.6883 - val_accuracy: 0.3843
Epoch 3/10
700/700 [==============================] - 183s 261ms/step - loss: 2.7527 - accuracy: 0.3886 - val_loss: 3.2668 - val_accuracy: 0.3945
Epoch 4/10
700/700 [==============================] - 183s 261ms/step 

In [0]:
predictions = model.predict(x)

In [0]:
tp = 0
tn = 0
fp = 0
fn = 0
for i in range(len(y)):
    for j in range(len(y[i])):
        actual = round(y[i][j][0])
        pred = round(predictions[i][j][0])
        if actual == 0 and pred == 0:
            tn += 1
        elif actual == 0 and pred > 0:
            fp += 1
        elif actual > 0 and pred == 0:
            fn += 1
        else:
            tp += 1
total = tp+tn+fp+fn

In [0]:
print("Accuracy:",(tp+tn)*100/total)
print("True Positive Rate:",tp*100/total)
print("True Negative Rate:",tn*100/total)
print("False Postiive Rate:",fp*100/total)
print("False Negative Rate:",fn*100/total)

Accuracy: 94.81222
True Positive Rate: 74.38024
True Negative Rate: 20.43198
False Postiive Rate: 4.37106
False Negative Rate: 0.81672
